In [1]:
# 모듈 import
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchFrameException, NoSuchElementException
from bs4 import BeautifulSoup
import time 

## 크롤링하고자 하는 음식 카테고리 지정
- 네이버지도 검색에 '00역 음식점', '00역 맛집' 등 검색 시 등장하는 음식 카테고리 크롤링

### 음식 카테고리 크롤링

In [ ]:
# 검색 결과가 표시되는 iframe으로 전환
def search_iframe():
    driver.switch_to.default_content() # 차상위 (기본) 프레임으로 전환
    driver.switch_to.frame("searchIframe") # 검색 결과 프레임으로 전환

def entry_iframe():
    driver.switch_to.default_content() # 차상위 (기본) 프레임으로 전환
    WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="entryIframe"]'))) # WebDriverWait : entryIframe이 나타날 때까지 대기

    # for 문으로 총 5번까지 시도, 0.5초간 대기 후 프레임 전환을 시도 -> frame 전환에 성공하면 break 문으로 빠져나온다
    for i in range(5):
        time.sleep(1.5)
        try:
            driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]'))
            break
        except:
            pass

# 화면에 표시된 업체명을 확인하는 함수
def chk_names():
    search_iframe() # search 프레임으로 전환
    elem = driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/a[1]/div/div/span[1]') # 해당 프레임에서 Xpath를 사용하여 업체명이 표시된 요소를 찾음 원래는 //*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/div/a[1]/div/div/span[1]
    name_list = [e.text for e in elem] # find_elements 함수로 업체명이 표시된 요소를 모두 찾고, 그 중 text 값을 가져와서 리스트에 저장

    return elem, name_list


def chk_details():
    entry_iframe() # 세부 정보의 iframe으로 전환
    soup = BeautifulSoup(driver.page_source, 'html.parser')
#     print(soup)
    # 데이터 쌓기 - 업종, 주소, url 순
    try:
        category = soup.find('span', {'class': 'lnJFt'})
    except:
        category = float('nan')


    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        ID = float('nan')

    search_iframe() # 다시 검색 결과의 iframe으로 전환

    return category, url, ID

def crawling_main():
    global naver_res # 전역변수 naver_res 사용 (크롤링 데이터를 저장하는 데이터프레임)
    category_list = [] # 업종 리스트
    url_list = [] # url 리스트
    ID_list = []

    for e in elem:
        try:
            e.click()
            category, url, ID = chk_details() # 업체의 세부 정보 가져오기
        except:
            category = ''
            url=''
            ID =''
        

        # 가져온 정보를 리스트에 추가
        category_list.append(category)
        url_list.append(url)
        ID_list.append(ID)

    naver_temp = pd.DataFrame([name_list, category_list, url_list, ID_list], index = naver_res.columns).T
    naver_res = pd.concat([naver_res, naver_temp])
    naver_res.to_excel('./naver_crawling_result.xlsx')



# webdriver 실행
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
keyword = '신촌역 맛집'
url = f'https://map.naver.com/p/search/{keyword}'
driver.get(url)
# 마우스 및 키보드 동작 시뮬레이터
action = ActionChains(driver)
# 불러올 정보를 담을 빈 데이터프레임 생성
naver_res = pd.DataFrame(columns=['업체명','업종','URL','ID'])

last_name = ''

page_num = 1

while True:
    time.sleep(1.5)
    search_iframe()
    elem, name_list = chk_names()
    if last_name == name_list[-1]:
        break
    while True:
        # 자동 스크롤 구현부
        driver.execute_script("arguments[0].scrollIntoView(true);", elem[-1])
        time.sleep(1.5)
        elem, name_list = chk_names()
        if last_name == name_list[-1]:
            break
        else:
            last_name = name_list[-1]
    crawling_main()
    # 다음 페이지로 이동
    try:
        next_button = driver.find_elements(By.CLASS_NAME, 'eUTV2')[-1]
        next_button.click()
    except:
        break

### 검색 시 등장한 음식 카테고리 확인

In [ ]:
set(naver_res['업종'].apply(lambda x: x.text))

### 크롤링 대상 음식 카테고리 지정

In [2]:
target_categories = ['게요리', '곱창,막창,양', '과일,주스전문점', '국밥', '국수', '김밥', '낙지요리', '냉면', 
    '다이어트,샐러드', '닭갈비', '닭발', '도시락,컵밥', '돈가스', '돼지고기구이', '두부요리', 
    '딤섬,중식만두', '떡볶이', '만두', '매운탕,해물탕', '맥주,호프', '바(BAR)', '베이글', '베이커리', 
    '베트남음식', '북카페', '분식', '브런치카페', '샌드위치', '생선회', '샤브샤브', '소고기구이', 
    '스테이크,립', '스파게티,파스타전문', '아시아음식', '양갈비', '양꼬치', '양식', '오뎅,꼬치', 
    '와인', '와플', '요리주점', '우동,소바', '육류,고기요리', '이자카야', '이탈리아음식', '인도음식', 
    '일식당', '일식튀김,꼬치', '장어,먹장어요리', '조개요리', '족발,보쌈', '종합분식', '주꾸미요리', 
    '중식당', '찌개,전골', '초밥,롤', '치킨,닭강정', '카페', '카페,디저트', '칼국수,만두', '케이크전문', 
    '태국음식', '테이크아웃커피', '토스트', '포장마차', '퓨전음식', '프랑스음식', '피자', '한식', 
    '한식뷔페', '한정식-일반', '해물,생선요리', '해장국', '햄버거', '감자탕', '곰탕,설렁탕', '기사식당', 
    '덮밥', '라면', '문래돼지불백', '백반,가정식', '백숙,삼계탕', '복어요리', '빙수', '순대,순댓국', 
    '아이스크림', '오므라이스', '죽', '찜닭', '추어탕', '카레', '패밀리레스토랑', '향토음식', 
    '홍차전문점', '후렌치후라이', '호떡', '쌈밥', '아귀찜,해물찜', '닭볶음탕', '대게요리', '마라탕', 
    '멕시코,남미음식', '브런치', '스페인음식', '정육식당', '전통,민속주점', '일본식라면']

## 크롤링하고자 하는 동 지정 (ex_ 아현동)

In [5]:
# 각자 동에 맞는 거로!
restaurant = pd.read_csv("./서울시마포구일반음식점인허가정보.csv")
restaurant = restaurant[restaurant['영업상태명'] == '영업/정상']

# NaN 값을 빈 문자열로 대체, 안 하면 에러남
restaurant['지번주소'] = restaurant['지번주소'].fillna('')


# '지번주소' 열이 '서울특별시 마포구 --동'으로 시작하는 행만 선택
target_dong = '서울특별시마포구연남동'
#dong = restaurant[restaurant['지번주소'].str.startswith(target_dong)]
dong = restaurant[restaurant['지번주소'].str.replace(' ', '').str.startswith(target_dong)]

# dong에서 도로명주소를 기준으로 검색할 것이기 때문에, 주소 기준 중복인 행을 다 없앰
dong = dong.drop_duplicates(subset='도로명주소', keep=False)
dong.reset_index(drop=True, inplace=True)
len(dong) # 해당 동의 음식점 개수

C:\Users\uje10\AppData\Local\Temp\ipykernel_22920\3753707766.py:2: DtypeWarning: Columns (41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant = pd.read_csv("./서울시마포구일반음식점인허가정보.csv")


796

## 크롤링하고자 하는 동 내 음식점 ID 추출

In [6]:
# webdriver 실행
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver = webdriver.Chrome()
# 검색 결과가 표시되는 iframe으로 전환
def search_iframe():
    driver.switch_to.default_content()  # 차상위 (기본) 프레임으로 전환
    try:
        time.sleep(1.5)  # 페이지 로딩 대기
        driver.switch_to.frame("searchIframe")  # 검색 결과 프레임으로 전환
    except NoSuchElementException:
        print("searchIframe을 찾을 수 없습니다.")
        return False
    return True

# 개별 업체의 세부 정보가 표시되는 iframe으로 전환
def entry_iframe():
    driver.switch_to.default_content()  # 차상위 (기본) 프레임으로 전환
    try:
        time.sleep(1.5)  # 페이지 로딩 대기
        driver.switch_to.frame("entryIframe")  # 세부 정보 프레임으로 전환
    except NoSuchElementException:
        print("entryIframe을 찾을 수 없습니다.")
        return False
    return True

# type1의 화면에 표시된 업체명을 확인하는 함수
def chk_names_type1():
    search_iframe()  # search 프레임으로 전환
    elems = driver.find_elements(By.CLASS_NAME, 'place_bluelink.C6RjW')
    name_list = []
    category_list = []
    elements = []
    
    for e in elems:
        try:
            name = e.find_element(By.CLASS_NAME, 'YwYLL').text
            category = e.find_element(By.CLASS_NAME, 'YzBgS').text
            
            # 업종이 target_categories에 포함되지 않으면 패스
            if category not in target_categories:
                continue
                
            name_list.append(name)
            category_list.append(category)
            elements.append(e)
        except:
            continue
            
    return elements, name_list, category_list

# type2의 화면에 표시된 업체명을 확인하는 함수
def chk_names_type2(processed_names):
    elems = driver.find_elements(By.CLASS_NAME, 'search_box')
    name_list = []
    category_list = []
    elements = []
    
    for e in elems:
        try:
            name = e.find_element(By.CLASS_NAME, 'search_title').text
            category = e.find_element(By.CLASS_NAME, 'category').text
            
            # 이미 처리된 이름은 패스
            if name in processed_names:
                continue
            
            # 업종이 target_categories에 포함되지 않으면 패스
            if category not in target_categories:
                continue
                
            name_list.append(name)
            category_list.append(category)
            elements.append(e)
        except:
            continue
            
    return elements, name_list, category_list

# 세부 페이지에서 URL과 ID를 추출하는 함수 (type1)
def chk_details():
    if not entry_iframe():  # 세부 정보의 iframe으로 전환
        return float('nan'), float('nan')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        url = float('nan')
        ID = float('nan')

    search_iframe()  # 다시 검색 결과의 iframe으로 전환
    return url, ID

# 세부 페이지에서 URL과 ID를 추출하는 함수 (type2)
def chk_details_type2():
    if not entry_iframe():  # 세부 정보의 iframe으로 전환
        return float('nan'), float('nan')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        url = float('nan')
        ID = float('nan')

    return url, ID

# 크롤링 메인 함수
def crawling_main(type):
    global naver_res  # 전역변수 naver_res 사용 (크롤링 데이터를 저장하는 데이터프레임)
    global target_dong
    
    processed_names = set()  # 이미 처리된 업체명을 추적하기 위한 집합
    
    if type == "type1":
        elements, name_list, category_list = chk_names_type1()
        details_func = chk_details
        
        id_list = []
        url_list = []

        for e in elements:
            try:
                e.click()  # 업체명 클릭
                time.sleep(1.5)  # 페이지 로딩 대기
                url, ID = details_func()  # 업체의 세부 정보 가져오기
                url_list.append(url)
                id_list.append(ID)

            except:
                url_list.append(float('nan'))
                id_list.append(float('nan'))
                continue
        
        # 새로운 데이터프레임 생성
        naver_temp = pd.DataFrame({
            '지번주소': [current_jibun] * len(name_list),
            '도로명주소': [current_keyword] * len(name_list),
            '업체명': name_list,
            '업종': category_list,
            'URL': url_list,
            'ID': id_list
        })
        
        # 기존 데이터프레임과 결합
        naver_res = pd.concat([naver_res, naver_temp])
        naver_res.reset_index(drop=True, inplace=True)
        naver_res.to_csv(f'./{target_dong.split()[-1]}.csv')  # 결과를 csv 파일로 저장
        
    if type == "type2":
        all_name_list = []
        all_category_list = []
        all_id_list = []
        all_url_list = []
        
        while True:
            elements, name_list, category_list = chk_names_type2(processed_names)
            details_func = chk_details_type2

            if len(elements) == 0:
                break

            for e, name, category in zip(elements, name_list, category_list):
                try:
                    e.click()  # 업체명 클릭
                    time.sleep(1.5)  # 페이지 로딩 대기
                    url, ID = details_func()  # 업체의 세부 정보 가져오기
                    all_url_list.append(url)
                    all_id_list.append(ID)
                    processed_names.add(name)  # 처리된 이름을 추가
                    all_name_list.append(name)
                    all_category_list.append(category)
                    driver.back()  # 뒤로가기 버튼 클릭
                    time.sleep(1.5)  # 뒤로가기 후 페이지 로딩 대기
                except:
                    continue
        
        # 새로운 데이터프레임 생성
        naver_temp = pd.DataFrame({
            '지번주소': [current_jibun] * len(all_name_list),
            '도로명주소': [current_keyword] * len(all_name_list),
            '업체명': all_name_list,
            '업종': all_category_list,
            'URL': all_url_list,
            'ID': all_id_list
        })

        # 기존 데이터프레임과 결합
        naver_res = pd.concat([naver_res, naver_temp])
        naver_res.to_csv(f'./{target_dong.split()[-1]}.csv')  # 결과를 csv 파일로 저장

# 불러올 정보를 담을 빈 데이터프레임 생성
# naver_res = pd.DataFrame(columns=['지번주소', '도로명주소', '업체명', '업종', 'URL', 'ID'])
naver_res = pd.read_csv('서울특별시마포구서교동.csv')


for index, row in dong[452:].iterrows():
    current_jibun = row['지번주소']  # '지번주소'를 저장
    current_keyword = row['도로명주소']  # '도로명주소'를 검색 키워드로 사용
    
    print(f'전체 {len(dong)} 중 {index+1}, 주소:{current_jibun}')
    
    url = f'https://map.naver.com/p/search/{current_keyword}'
    driver.get(url)

    # "sc-1wsjitl dunggE overlap" 클래스가 있는지 확인
    try:
        time.sleep(1.5)  # 페이지 로딩 대기
        overlap_element = driver.find_element(By.CLASS_NAME, 'sc-1wsjitl.dunggE.overlap')
        try:
            more_button = overlap_element.find_element(By.CLASS_NAME, 'link_more')
            more_button.click()
            time.sleep(1.5)  # "더보기" 버튼 클릭 후 페이지 로딩 대기
        except NoSuchElementException:
            pass  # "더보기" 버튼이 없는 경우 그냥 넘어감
        crawling_main("type2")
    except NoSuchElementException:
        try:
            no_result = driver.find_element(By.CLASS_NAME, 'correction_result_text')
            continue
        except:
            try:
                crawling_main("type1")
            except NoSuchElementException:
                continue

    action = ActionChains(driver)
driver.quit()

전체 796 중 1, 주소:서울특별시 마포구 연남동 227-1번지 1층 
전체 796 중 2, 주소:서울특별시 마포구 연남동 228-1
전체 796 중 3, 주소:서울특별시 마포구 연남동 568-23 (1층)
전체 796 중 4, 주소:서울특별시 마포구 연남동 260-28 1층
전체 796 중 5, 주소:서울특별시 마포구 연남동 509-14번지 1층 
전체 796 중 6, 주소:서울특별시 마포구 연남동 228-5 
전체 796 중 7, 주소:서울특별시 마포구 연남동 566-8번지 1층 
전체 796 중 8, 주소:서울특별시 마포구 연남동 224-19
전체 796 중 9, 주소:서울특별시 마포구 연남동 487-366번지 
전체 796 중 10, 주소:서울특별시 마포구 연남동 561-2
전체 796 중 11, 주소:서울특별시 마포구 연남동 228-26번지 
전체 796 중 12, 주소:서울특별시 마포구 연남동 259-5
전체 796 중 13, 주소:서울특별시 마포구 연남동 228-40
전체 796 중 14, 주소:서울특별시 마포구 연남동 228-42 
전체 796 중 15, 주소:서울특별시 마포구 연남동 487-412 
전체 796 중 16, 주소:서울특별시 마포구 연남동 562-27번지 (1층) 
전체 796 중 17, 주소:서울특별시 마포구 연남동 561-8번지 (1층) 
전체 796 중 18, 주소:서울특별시 마포구 연남동 562-26
전체 796 중 19, 주소:서울특별시 마포구 연남동 258-14 
전체 796 중 20, 주소:서울특별시 마포구 연남동 493-31번지 
전체 796 중 21, 주소:서울특별시 마포구 연남동 487-71 
전체 796 중 22, 주소:서울특별시 마포구 연남동 487-226
전체 796 중 23, 주소:서울특별시 마포구 연남동 226-29번지 지하1층 
전체 796 중 24, 주소:서울특별시 마포구 연남동 361-11번지 1층 
전체 796 중 25, 주소:서울특별시 마포구 연남동 487-241번지 
전체 796 중 26, 주

전체 796 중 198, 주소:서울특별시 마포구 연남동 515-20번지 
전체 796 중 199, 주소:서울특별시 마포구 연남동 260-52 2,3층 
전체 796 중 200, 주소:서울특별시 마포구 연남동 260-52번지 지하1층 
전체 796 중 201, 주소:서울특별시 마포구 연남동 566-55
전체 796 중 202, 주소:서울특별시 마포구 연남동 383-99 
전체 796 중 203, 주소:서울특별시 마포구 연남동 228-2번지 2층우측 
전체 796 중 204, 주소:서울특별시 마포구 연남동 228-47번지 
전체 796 중 205, 주소:서울특별시 마포구 연남동 390-73번지 1층 
전체 796 중 206, 주소:서울특별시 마포구 연남동 390-51
전체 796 중 207, 주소:서울특별시 마포구 연남동 260-51 2층
전체 796 중 208, 주소:서울특별시 마포구 연남동 260-31번지 
전체 796 중 209, 주소:서울특별시 마포구 연남동 570-28번지 
전체 796 중 210, 주소:서울특별시 마포구 연남동 260-45 지1층
전체 796 중 211, 주소:서울특별시 마포구 연남동 228-44 1층일부
전체 796 중 212, 주소:서울특별시 마포구 연남동 229-34 
전체 796 중 213, 주소:서울특별시 마포구 연남동 260-52 
전체 796 중 214, 주소:서울특별시 마포구 연남동 361-10번지 
전체 796 중 215, 주소:서울특별시 마포구 연남동 384-11
전체 796 중 216, 주소:서울특별시 마포구 연남동 361-10번지 3층 
전체 796 중 217, 주소:서울특별시 마포구 연남동 383-95
전체 796 중 218, 주소:서울특별시 마포구 연남동 241-98
전체 796 중 219, 주소:서울특별시 마포구 연남동 385-6
전체 796 중 220, 주소:서울특별시 마포구 연남동 387-7번지 
전체 796 중 221, 주소:서울특별시 마포구 연남동 390-78번지 
전체 796 중 222, 주소:서울특별

전체 796 중 401, 주소:서울특별시 마포구 연남동 228-42
전체 796 중 402, 주소:서울특별시 마포구 연남동 229-67 
전체 796 중 403, 주소:서울특별시 마포구 연남동 454-7 
전체 796 중 404, 주소:서울특별시 마포구 연남동 390-100
전체 796 중 405, 주소:서울특별시 마포구 연남동 390-71 
전체 796 중 406, 주소:서울특별시 마포구 연남동 241-86 
전체 796 중 407, 주소:서울특별시 마포구 연남동 387-26
전체 796 중 408, 주소:서울특별시 마포구 연남동 228-55 
전체 796 중 409, 주소:서울특별시 마포구 연남동 387-31 
전체 796 중 410, 주소:서울특별시 마포구 연남동 566-60
전체 796 중 411, 주소:서울특별시 마포구 연남동 383-95 
전체 796 중 412, 주소:서울특별시 마포구 연남동 228-21
전체 796 중 413, 주소:서울특별시 마포구 연남동 260-46 
전체 796 중 414, 주소:서울특별시 마포구 연남동 383-78 
전체 796 중 415, 주소:서울특별시 마포구 연남동 228-8
전체 796 중 416, 주소:서울특별시 마포구 연남동 224-18
전체 796 중 417, 주소:서울특별시 마포구 연남동 240-38
전체 796 중 418, 주소:서울특별시 마포구 연남동 384-7 
전체 796 중 419, 주소:서울특별시 마포구 연남동 223-111 동보빌딩
전체 796 중 420, 주소:서울특별시 마포구 연남동 227-22 대원빌딩 
전체 796 중 421, 주소:서울특별시 마포구 연남동 228-2
전체 796 중 422, 주소:서울특별시 마포구 연남동 384-8 
전체 796 중 423, 주소:서울특별시 마포구 연남동 387-8
전체 796 중 424, 주소:서울특별시 마포구 연남동 383-82 신박사빌딩 
전체 796 중 425, 주소:서울특별시 마포구 연남동 390-35 지빌딩 
전체 796 중 426, 주소:서울특

전체 796 중 610, 주소:서울특별시 마포구 연남동 257-22
전체 796 중 611, 주소:서울특별시 마포구 연남동 223-28
전체 796 중 612, 주소:서울특별시 마포구 연남동 223-50
전체 796 중 613, 주소:서울특별시 마포구 연남동 390-59
전체 796 중 614, 주소:서울특별시 마포구 연남동 257-18
전체 796 중 615, 주소:서울특별시 마포구 연남동 478-18 만민하늘애
전체 796 중 616, 주소:서울특별시 마포구 연남동 241-90
전체 796 중 617, 주소:서울특별시 마포구 연남동 373-14
전체 796 중 618, 주소:서울특별시 마포구 연남동 509-29
전체 796 중 619, 주소:서울특별시 마포구 연남동 566-12
전체 796 중 620, 주소:서울특별시 마포구 연남동 382-27 27번지
전체 796 중 621, 주소:서울특별시 마포구 연남동 387-4 維豪-위호
전체 796 중 622, 주소:서울특별시 마포구 연남동 390-74
전체 796 중 623, 주소:서울특별시 마포구 연남동 383-34
전체 796 중 624, 주소:서울특별시 마포구 연남동 373-17
전체 796 중 625, 주소:서울특별시 마포구 연남동 567-6
전체 796 중 626, 주소:서울특별시 마포구 연남동 228-58
전체 796 중 627, 주소:서울특별시 마포구 연남동 487-287
전체 796 중 628, 주소:서울특별시 마포구 연남동 383-34
전체 796 중 629, 주소:서울특별시 마포구 연남동 570-23
전체 796 중 630, 주소:서울특별시 마포구 연남동 228-5
전체 796 중 631, 주소:서울특별시 마포구 연남동 390-84
전체 796 중 632, 주소:서울특별시 마포구 연남동 228-23
전체 796 중 633, 주소:서울특별시 마포구 연남동 383-90
전체 796 중 634, 주소:서울특별시 마포구 연남동 225-35 MH빌딩
전체 796 중 635, 주소:서울특별시 마포구 연남동

## 중복행 제거 및 저장

In [7]:
# 중복행 제거
naver_res_drop = naver_res.drop_duplicates(subset='ID', keep='first')

In [8]:
# CSV 파일로 저장
naver_res_drop.to_csv('서교동.csv', index=False, encoding = 'utf-8-sig')